In [25]:
#Change these depending on the file path
group = 'recovered_021'

read_folder = group+'_EMG'
output_folder_path = group+'_resampled'

In [26]:
# import libraries
import os
import numpy as np 
import pandas as pd 
from scipy.signal import resample 
import matplotlib.pyplot as plt 

In [27]:
#set the working directory
os.chdir('/Users/rossbrancati/Desktop/PFP_Project/EMG/Data/V3D_data/')
data_directory = os.path.join(os.getcwd(), read_folder)
directory = sorted(os.listdir(data_directory))
#directory.remove('.DS_Store')
print(directory)

#make a new directory to store the PCA matrices in
pca_directory = os.path.join(os.getcwd(),'PCA_matrices')

['S14_EMG_021.txt', 'S15_EMG_021.txt', 'S28_EMG_021.txt', 'S30_EMG_021.txt', 'S31_EMG_021.txt', 'S33_EMG_021.txt', 'S35_EMG_021.txt', 'S37_EMG_021.txt', 'S42_EMG_021.txt', 'S43_EMG_021.txt', 'S46_EMG_021.txt', 'S48_EMG_021.txt', 'S49_EMG_021.txt', 'S50_EMG_021.txt', 'S59_EMG_021.txt']


In [28]:
#Create an empty matrix for each muscle that will be appended with each new sheet. This is 1 column of 101 zeros
biceps_femoris = np.zeros([101,1])
glute_max = np.zeros([101,1])
glute_med = np.zeros([101,1])
lateral_gastroc = np.zeros([101,1])
medial_gastroc = np.zeros([101,1])
rectus_femoris = np.zeros([101,1])
soleus = np.zeros([101,1])
semitendinosus = np.zeros([101,1])
tibialis_anterior = np.zeros([101,1])
vastus_lateralis = np.zeros([101,1])
vastus_medialis = np.zeros([101,1])

#Create a list that contains each of the empty matrices for looping over to store data
pca_file_list = [biceps_femoris, glute_max, glute_med, lateral_gastroc, medial_gastroc, rectus_femoris, 
                 soleus, semitendinosus, tibialis_anterior, vastus_lateralis, vastus_medialis]

#create a list of strings for exporting files
pca_file_list_names = ['biceps_femoris', 'glute_max', 'glute_med', 'lateral_gastroc', 'medial_gastroc', 'rectus_femoris', 
                 'soleus', 'semitendinosus', 'tibialis_anterior', 'vastus_lateralis', 'vastus_medialis']

In [29]:
#Loop through each file 
total_strides = 0

for file_idx,file_name in enumerate(directory):
    
    #set variable as file path
    file_path = os.path.join(data_directory,file_name)
    
    #load the file
    raw_file = pd.read_csv(file_path,sep='\t',skiprows=4)
    _,num_cols = raw_file.shape
    
    #preallocate a matrix to store the resampled data in
    resampled_matrix = np.zeros([101,num_cols-1])
    
    #loop through each column in the raw_file and resample to 101 data points
    #list of the varaibles which represent each column
    column_list=raw_file.columns
    
    #loop through each column
    for col_idx,column_id in enumerate(column_list[1:]):
        
        #get the current column as a vector
        current_col = np.array(raw_file.loc[:,column_id].dropna())
        
        #resample to 101 data points
        resamp_col = resample(current_col,101)
        
        #store resamples vector in resampled matrix
        resampled_matrix[:,col_idx] = resamp_col
        
        
    #stride counter - used for plotting purposes
    stride_count = int((num_cols-1)/11)
    #print(stride_count)
    
    #count the total number of strides
    total_strides = total_strides + stride_count
    print(total_strides)
    
    #append muscle matrices with data from new sheet
    #biceps_femoris = np.hstack((biceps_femoris,resampled_matrix[:,0:stride_count]))
    #glute_max = np.hstack((glute_max,resampled_matrix[:,stride_count:stride_count*2]))
    #glute_med = np.hstack((glute_med,resampled_matrix[:,stride_count*2:stride_count*3]))
    #lateral_gastroc = np.hstack((lateral_gastroc,resampled_matrix[:,stride_count*3:stride_count*4]))
    #medial_gastroc = np.hstack((medial_gastroc,resampled_matrix[:,stride_count*4:stride_count*5]))
    #rectus_femoris = np.hstack((rectus_femoris,resampled_matrix[:,stride_count*5:stride_count*6]))
    #soleus = np.hstack((soleus,resampled_matrix[:,stride_count*6:stride_count*7]))
    #semitendinosus = np.hstack((semitendinosus,resampled_matrix[:,stride_count*7:stride_count*8]))
    #tibialis_anterior = np.hstack((tibialis_anterior,resampled_matrix[:,stride_count*8:stride_count*9]))
    #vastus_lateralis = np.hstack((vastus_lateralis,resampled_matrix[:,stride_count*9:stride_count*10]))
    #vastus_medialis = np.hstack((vastus_medialis,resampled_matrix[:,stride_count*10:stride_count*11]))
    
    for i in range(11):
        pca_file_list[i] = np.hstack((pca_file_list[i],resampled_matrix[:,i*stride_count:(i+1)*stride_count]))

13
20
32
44
52
64
75
87
95
107
118
130
140
153
167


In [30]:
#remove first column of zeros from each of the muscle matrices and export file to csv
for ii in range(11):
    pca_file_list[ii] = pca_file_list[ii][:,1:]
    current_file_id = os.path.join('/Users/rossbrancati/Desktop/PFP_Project/EMG/Data/resampled_muscle_matrices/',output_folder_path,pca_file_list_names[ii])
    np.savetxt(current_file_id+'.txt', pca_file_list[ii], delimiter="\t")